In [61]:
from pyzbar.pyzbar import decode
from pdf2image import convert_from_path
import os
import numpy as np
from datetime import datetime, timedelta

In [36]:
# aqui buscamos dentro do caminho somente os arquivos .pdf, e no caso so permitira dentro da lista arquivos que sejam .pdf
pdfs = [i for i in os.listdir() if '.pdf' in i]
pdf_path = pdfs[0]
display(pdf_path)

'boleto.pdf'

In [42]:
# aqui vamos converter um arquivo pdf em formato de imagem
pages = convert_from_path(pdf_path)
img = pages[0]

In [ ]:
def conversao_vencimento(dias_calc):
    date_format = "%d/%m/%Y"
    date_string = "7/10/1997"
    date_obj = datetime.strptime(date_string, date_format).date()
    new_date = date_obj + timedelta(days=dias_calc)

    return new_date.strftime(date_format)

In [83]:
def teste():
    pesos10 = [1, 2, 1, 2, 1, 2, 1, 2, 1, 2]
    segmentos = ['8400004690']

    for segmento in segmentos:
        calculo_valores = [int(campo) * peso for campo,
                           peso in zip(segmento, pesos10)]
        soma_campos = sum(calculo_valores)
        soma_campos = soma_campos % 10
        if soma_campos == 0:
            soma_campos = 0
        else:
            soma_campos = 10 - soma_campos
        soma_campos = soma_campos - 1

    print(soma_campos)


teste()

8


In [118]:
def calcular_digito_verificador_modulo_10(codigo):
    # Passo 1: Inverte a sequência dos 9 números do código
    codigo_invertido = codigo[::-1]

    soma = 0
    multiplicador = 2

    for digito in codigo_invertido:
        # Passo 2: Multiplica os dígitos alternadamente por 2 e por 1
        resultado = int(digito) * multiplicador

        # Passo 2 (continuação): Se o resultado for maior que 9, soma os dígitos
        if resultado > 9:
            resultado = resultado // 10 + resultado % 10

        # Passo 3: Soma os resultados das multiplicações
        soma += resultado

        # Troca o multiplicador para alternar entre 1 e 2
        multiplicador = 3 - multiplicador

    # Passo 4: Encontra o múltiplo de 10 mais próximo da soma
    multiplo_dez_proximo = (soma // 10 + 1) * 10

    # Passo 5: Calcula o dígito verificador
    digito_verificador = multiplo_dez_proximo - soma

    return digito_verificador

In [119]:
def calcular_digito_verificador_modulo_11(codigo):
    # Passo 1: Inverte a sequência dos 9 números do código
    codigo_invertido = codigo[::-1]

    soma = 0
    peso = 2

    for digito in codigo_invertido:
        # Passo 2 e 3: Multiplica o dígito pelo peso correspondente
        resultado = int(digito) * peso

        # Passo 4: Soma os resultados das multiplicações
        soma += resultado

        # Incrementa o peso, mantendo-o entre 2 e 9 (circularmente)
        peso = peso % 9 + 1

    # Passo 5: Calcula o resto da divisão por 11
    resto = soma % 11

    # Passo 6: Calcula o dígito verificador
    if resto in (0, 1):
        digito_verificador = 0
    else:
        digito_verificador = 11 - resto

    return digito_verificador

In [121]:
def calcular_digito_verificador_modulo_11_geral(codigo):
    # Passo 1: Concatena os três campos do código de barras
    codigo_completo = codigo[0:9] + codigo[10:20] + codigo[21:31]

    soma = 0
    peso = 2

    for digito in codigo_completo[::-1]:
        # Passo 2 e 3: Multiplica o dígito pelo peso correspondente
        resultado = int(digito) * peso

        # Passo 4: Soma os resultados das multiplicações
        soma += resultado

        # Incrementa o peso, mantendo-o entre 2 e 9 (circularmente)
        peso = peso % 9 + 1

    # Passo 5: Calcula o resto da divisão por 11
    resto = soma % 11

    # Passo 6: Calcula o dígito verificador
    if resto in (0, 1):
        digito_verificador = 1
    else:
        digito_verificador = 11 - resto

    return digito_verificador

In [130]:
def detectando_cod_bars():
    detected_datas = decode(img)
    if not detected_datas:
        return False
    else:
        for barcode in detected_datas:
            barcode = detected_datas[0]
            if barcode.data != '' and barcode.type == 'I25':
                # retornar a uma string do python
                barcode_ = barcode.data.decode('utf-8')
    return barcode_


def calculo_codbar(barcode):
    if barcode[0] != '8':
        barcode_geral = barcode
        cod_banco = barcode[:3]
        cod_moeda = barcode[3]
        cod_verificador = barcode[4]
        fator_venc = barcode[5:9]
        valor_bol = barcode[9:19]
        campo_livre_1 = barcode[19:24]
        campo_livre_2 = barcode[24:34]
        campo_livre_3 = barcode[34:44]

        campo1 = cod_banco + cod_moeda + campo_livre_1
        campo2 = campo_livre_2
        campo3 = campo_livre_3
        calculo_dv_1 = calcular_digito_verificador_modulo_10(campo1)
        calculo_dv_2 = calcular_digito_verificador_modulo_10(campo2)
        calculo_dv_3 = calcular_digito_verificador_modulo_10(campo3)
        calculo_dv_geral = calcular_digito_verificador_modulo_11_geral(
            barcode_geral)

        fator_venc_data = conversao_vencimento(int(fator_venc))

        barcode_complete = f'{campo1}{calculo_dv_1}{campo2}{calculo_dv_2}{campo3}{calculo_dv_3}{calculo_dv_geral}{fator_venc}{valor_bol}'

        return barcode_complete


teste = detectando_cod_bars()
print(teste)
calculo_codbar(teste)

23791932700000124751229260012262678400004690


'23791229286001226267584000046908193270000012475'

In [150]:
# decodificando foto e pegando os dados
# podemos verificar o tipo do código de barras, usaremos .type

def calculo_codbar(barcode):
    if barcode[0] != '8':
        barcode_geral = barcode
        cod_banco = barcode[:3]
        cod_moeda = barcode[3]
        cod_verificador = barcode[4]
        fator_venc = barcode[5:9]
        valor_bol = barcode[9:19]
        campo_livre_1 = barcode[19:24]
        campo_livre_2 = barcode[24:34]
        campo_livre_3 = barcode[34:44]

        campo1 = cod_banco + cod_moeda + campo_livre_1
        campo2 = campo_livre_2
        campo3 = campo_livre_3
        calculo_dv_1 = calcular_digito_verificador_modulo_10(campo1)
        calculo_dv_2 = calcular_digito_verificador_modulo_10(campo2)
        calculo_dv_3 = calcular_digito_verificador_modulo_10(campo3)
        calculo_dv_geral = calcular_digito_verificador_modulo_11_geral(
            barcode_geral)

        fator_venc_data = conversao_vencimento(int(fator_venc))

        barcode_complete = [
            f'{campo1[0:5]}.{campo1[5:9]}{calculo_dv_1}.{campo2[0:5]}.{campo2[5:10]}{calculo_dv_2}.{campo3[0:5]}.{campo3[5:10]}{calculo_dv_3}.{calculo_dv_geral}.{fator_venc}{valor_bol}']

        return barcode_complete

    if barcode[0] == '8':
        match barcode[1]:
            case '1':
                print('Prefeituras')
            case '2':
                print('Saneamento')
            case '3':
                print('Energia Elétrica e Gás')
            case '4':
                print('Telecomunicação')
            case '5':
                print('Órgãos Governamentais')
            case '6':
                print(
                    'Carnês e assemelhados ou demais empresas/orgãos id através do CNPJ')
            case '7':
                print('Multas de trânsito')
            case '9':
                print('Uso exclusivo do banco')
        match barcode[2]:
            case '6':
                print('Modulo 10')
            case '7':
                print('Modulo 10')
            case '8':
                print('Modulo 11')
            case '9':
                print('Modulo 11')

    if barcode[0] == '8' and barcode[2] in ['8', '9']:
        pesos = [4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]

        start = 0
        segmentos = []
        digitos = []
        barcode_complete = []

        for i in range(4):
            segmentos.append(barcode[start:start+11])
            start += 11

        for segmento in segmentos:
            calculo_valores = [int(campo) * peso for campo,
                               peso in zip(segmento, pesos)]
            soma_campos = sum(calculo_valores)
            soma_campos = soma_campos % 11
            if soma_campos in [0, 1]:
                soma_campos = 0
            elif soma_campos == 10:
                soma_campos = 1
            else:
                soma_campos = 11 - soma_campos

            digitos.append(soma_campos)

        for segmento, digito in zip(segmentos, digitos):
            combinacao_campos = segmento + str(digito)
            barcode_complete.append(combinacao_campos)

        return barcode_complete

    if barcode[0] != '8' and barcode[2] in ['6', '7']:
        pesos = [1, 2, 1, 2, 1, 2, 1, 2, 1, 2]

        start = 0
        segmentos = []
        digitos = []
        barcode_complete = []

        for i in range(4):
            segmentos.append(barcode[start:start+11])
            start += 11

        for segmento in segmentos:
            calculo_valores = [int(campo) * peso for campo,
                               peso in zip(segmento, pesos)]
            soma_campos = sum(calculo_valores)
            soma_campos = soma_campos % 10
            if soma_campos == 0:
                soma_campos = 0
            else:
                soma_campos = 10 - soma_campos

            digitos.append(soma_campos)

        for segmento, digito in zip(segmentos, digitos):
            combinacao_campos = segmento + str(digito)
            barcode_complete.append(combinacao_campos)

        return barcode_complete


barcode = '23791932700000124751229260012262678400004690'
calculo = calculo_codbar(barcode)
print(calculo)

['23791.22928.60012.262675.84000.046908.1.93270000012475']
